In [ ]:
# This allows changes in chakra.py to be automatically re-imported
# (this is tricky with OOP though, to be used with care)
%load_ext autoreload
%autoreload 1
%aimport chakra

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Get the default params
from oggm import cfg
cfg.initialize(logging_level='WORKFLOW')
cfg.PARAMS['cfl_number'] = 0.01  # less numerical instabilities
cfg.PARAMS['use_multiprocessing'] = False

In [ ]:
from oggm.core.massbalance import ScalarMassBalance
no_mb = ScalarMassBalance()

# BU bed with tributary

In [ ]:
bu_fls = chakra.bu_tidewater_bed(split_flowline_before_water=5)

xc = np.arange(bu_fls[0].nx + bu_fls[1].nx) * bu_fls[0].dx_meter / 1000
x_sep = (bu_fls[0].nx - 0.5) * bu_fls[0].dx_meter / 1000
f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(xc, np.append(bu_fls[0].bed_h, bu_fls[1].bed_h), color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.vlines(x_sep, -200, 600, color='C3', linestyles='-')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

# K-calving param with tributary

OGGM will work exactly the same with this tributary system (one can test for that). It is however a bit more code to plot it because it is all split in two:

In [ ]:
to_plot = None
    
one_model = chakra.KCalvingModel(chakra.bu_tidewater_bed(), mb_model=no_mb, 
                                 flux_gate=0.10,
                                 calving_use_limiter=True,
                                 do_kcalving=True,
                                 is_tidewater=True,
                                 calving_k=0.2,
                                )
    
two_model = chakra.KCalvingModel(bu_fls, mb_model=no_mb, 
                                 smooth_trib_influx=False,  # enforce that all mass comes into first point
                                 flux_gate=[0.10, 0],  # the default in OGGM is to add the fluxgate to each fl
                                 calving_use_limiter=True,
                                 do_kcalving=True,
                                 is_tidewater=True,
                                 calving_k=0.2,
                                )
    
# long enough to reach approx. equilibrium 
_, ds1 = one_model.run_until_and_store(6000)
_, ds2 = two_model.run_until_and_store(6000)

In [ ]:
ds1.volume_m3.plot();
ds2.volume_m3.plot(linestyle='--');

In [ ]:
df_diag = one_model.get_diagnostics(fl_id=0)
df_diag_1 = two_model.get_diagnostics(fl_id=0)
df_diag_2 = two_model.get_diagnostics(fl_id=1)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(xc, np.append(df_diag_1.surface_h, df_diag_2.surface_h), color='C0')
ax.plot(xc, df_diag.surface_h, color='C1', linestyle='--')
ax.plot(xc, np.append(bu_fls[0].bed_h, bu_fls[1].bed_h), color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.vlines(x_sep, -200, 600, color='C3', linestyles='-')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');